In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv(r"C:\Users\mariajose_chinchilla\Downloads\base_contagio.csv", sep=";")

In [3]:
df = df[(df["DxEntidad"] != "BANCO DE LOS TRABAJADORES") & (df["DxCategoriaSIRCMaxima"].isin(["B", "C", "D", "E"]))]

In [4]:
def es_categoria(val, cat):
    if val == cat:
        return 1
    elif val != cat:
        return 0

In [5]:
for cat in df["DxCategoriaSIRC"].unique().tolist():
    df[f"ES {cat}"] = df.apply(lambda x: es_categoria(x["DxCategoriaSIRC"], cat), axis=1)

In [6]:
df_agrupado = df.groupby(by="CODIGO_CLIENTE").agg(
 TOTAL_A=("ES A", "sum"),
 TOTAL_B=("ES B", "sum"),
 TOTAL_C=("ES C", "sum"),
 TOTAL_D=("ES D", "sum"),
 TOTAL_E=("ES E", "sum")   
).reset_index()

In [7]:
df_agrupado["EN RIESGO"] = df_agrupado["TOTAL_C"] + df_agrupado["TOTAL_D"] + df_agrupado["TOTAL_E"]
riegosos = df_agrupado["EN RIESGO"] >= 1
df_riesgosos = df_agrupado.loc[riegosos]

In [8]:
prop_con1 = len(df_riesgosos[df_riesgosos["EN RIESGO"] == 1]) / len(df_riesgosos)
prop_mas1 = len(df_riesgosos[df_riesgosos["EN RIESGO"] > 1]) / len(df_riesgosos)
print(f"Con exactamente 1 hay {prop_con1} y con más de uno hay {prop_mas1}")

Con exactamente 1 hay 0.9504814305364512 y con más de uno hay 0.04951856946354883


In [9]:
riesgosos = df[((df["ES C"] >= 1) | (df["ES D"] >= 1) | (df["ES E"] >= 1)) & (~df["DxCategoriaSIRC"].isna())]
dic_reservas = {"A": 0.036*0.45, "B": 0.086*0.45, "C": 0.156*0.45, "D": 0.325*0.45, "E": 1}

In [10]:
def poner_reservas(cat, dic):
    return dic.get(cat)

In [11]:
riesgosos[f"RESERVAS"] = riesgosos.apply(lambda x: poner_reservas(x["DxCategoriaSIRC"], dic_reservas), axis=1)

In [12]:
riesgosos["PAGO DE RESERVAS"] = riesgosos["RESERVAS"] * riesgosos["total_saldo"] 
riesgosos["PAGO DE RESERVAS"].sum()

9938152.119206

In [13]:
cat = ["C", "D", "E"]
for c in cat:
    total = riesgosos[riesgosos[f"ES {c}"] == 1]["total_saldo"].sum()
    print(f"En categoría {c} hay {total}")

En categoría C hay 84642111.28
En categoría D hay 14116839.639999999
En categoría E hay 1931688.11


In [14]:
riesgosos["total_saldo"].sum()

100690639.03

In [15]:
riesgosos["DfSaldo"].sum()

25408147.11

In [21]:
riesgosos_sorted = riesgosos.sort_values(by=['codigo_ejecutivo', 'total_saldo'], ascending=[True, False])

# Crear una columna temporal 'grupo_prioridad' para agrupar por 'codigo_ejecutivo' y 'DPI'
riesgosos_sorted['grupo_prioridad'] = riesgosos_sorted.groupby(['codigo_ejecutivo', 'DPI']).ngroup()

# Ordenar por 'codigo_ejecutivo' y 'grupo_prioridad' para mantener el orden deseado
riesgosos_sorted = riesgosos_sorted.sort_values(by=['codigo_ejecutivo', 'grupo_prioridad'])

# Asignar la prioridad dentro de cada grupo de 'codigo_ejecutivo'
riesgosos_sorted['PRIORIDAD ABF'] = riesgosos_sorted.groupby('codigo_ejecutivo')['grupo_prioridad'].rank(method='dense').astype(int)

# Mapear la prioridad a cada DPI dentro de cada 'codigo_ejecutivo'
priority_map = riesgosos_sorted.drop_duplicates(subset=['codigo_ejecutivo', 'DPI']).set_index(['codigo_ejecutivo', 'DPI'])['PRIORIDAD ABF'].to_dict()
riesgosos_sorted['PRIORIDAD ABF'] = riesgosos_sorted.set_index(['codigo_ejecutivo', 'DPI']).index.map(priority_map)

# Eliminar la columna temporal 'grupo_prioridad' ya que solo era para la ayuda en la agrupación
riesgosos_sorted = riesgosos_sorted.drop(columns=['grupo_prioridad'])

In [24]:
riesgosos_sorted.to_excel(r"C:\Users\mariajose_chinchilla\Documents\GitHub\proyectos_varios\Salidas\comerciales\Base de consolidación.xlsx")